IMPORT ALL THE REQUIRED LIBRARIES 

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse
import nltk
import os
import json
import math
import csv
import re
import time
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import joblib
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from collections import Counter
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD

DATA PREPROCESSING
READ THE DATASET AND NAME THE SELECTED COLUMNS IN CORRECT FORMAT

In [3]:
# DATE PREPROCESSING
# Accessing the dataset and picking 
elect_data = pd.read_csv('electronics_dataset.csv')


# Exploratory Data Analysis to show some information about the overall dataset

# view the first ten columns from the dataset
# elect_data.head(10)

# view the number of rows and columns in the dataset
# elect_data.shape

# view the datatypes of the data columns present in the dataset
# elect_data.dtypes()

# view the other information about the dataset
# elect_data.describe()
# elect_data.info()

# elect_data.head(10)



In [ ]:
#create a new datset with only the columns that are highly correlated and relevant to the analysis
new_df = elect_data[['id','brand','categories',"reviews.rating","reviews.text",'reviews.username']]
print(type(new_df))

# Exploratory Data Analysis to show some information about the new dataset

# view the first ten columns from the dataset
new_df.head(10)

# view the number of rows and columns in the dataset
# new_df.shape

# view the datatypes of the data columns present in the dataset
# new_df.dtypes

# view the other information about the dataset
# THE distribution of the ratings which are numerical varaibles
# new_df.describe()
# new_df.info()


THE DATASET HAS SOME NULL VALUES THAT ARE NOT IMPORTANT FOR OUR ANALYSIS. SO WE ANALYZE THE DISTRIBUTION OF THE NULL VALUES ACROSS THE COLUMNS 

In [ ]:
# check for null values in the dataset and store in a variable 
null_colls = new_df.isnull()
null_colls.sum()

# visializing the null values in the dataset with a heatmap
sns.heatmap(null_colls,yticklabels=False,cbar=False,cmap='viridis')



# Replacing the null values with the mean of the column
# for col in num_col:
#     elect_data[col] = pd.to_numeric(elect_data[col])
#     elect_data[col] = elect_data[col].fillna(elect_data[col].mean(), inplace=True)

# elect_data.head()

# Replace the null values with the 'NaN' string
# for col in null_col:
#     elect_data[col] = elect_data[col].fillna('NaN', inplace=True)

# print(elect_data.head(10))


# # Drop the columns with the highest number of null values as they are irrelevant to the analysis
# elect_data.drop(null_col, axis=1, inplace=True)

# print(elect_data.head(5))




# new_df.to_csv('new_electronics_dataset.csv', index=False)

VIEW THE DISTRIBUTION OF THE RATINGS ACROSS THE ELECTRONIC PRODUCTS

In [ ]:
sns.set_theme(style="darkgrid")

fig = sns.catplot(data=new_df, x="reviews.rating",aspect=2.0, kind='count')
fig.set_ylabels("Ratings across products")
    

In [ ]:
# plt.figure(figsize=(10,10))

# # create a correlation matrix of the dataset
# data_corr = df.corr()
# sns.heatmap(data_corr, annot=True, cmap="Blues")

WE CAN SEE THAT THE DISTRIBUTION OF THE RATINGS IS NOT EVEN. SOME PRODUCTS HAVE MORE RATINGS THAN OTHERS. THIS IS BECAUSE SOME USERS HAVE RATED MULTIPLE PRODUCTS AND VICE-VERSA SO WE NEED TO CHECK THE NUMBER OF UNIQUE USERS WHO HAVE RATED THE PRODUCTS AND ALSO THE NUMBER OF UNIQUE PRODUCTS THAT HAVE BEEN RATED BY THE USERS


In [79]:
# find the unique user reviews
unique_user_reviews = np.unique(new_df['reviews.username'])

# find the length of the unique user reviews
len_unique_user_reviews = len(unique_user_reviews)

print("The number of actual users in the dataset is: ", len_unique_user_reviews)

# find the unique products
unique_products = np.unique(new_df['id'])

# find the number of unique products
no_unique_products = len(unique_products)

print("The number of unique products in the dataset is: ", no_unique_products)




The number of actual users in the dataset is:  6493
The number of unique products in the dataset is:  50


TO TRAIN THE MODEL, THE REVIIEWS TEXT NEEDS TO BE PREPROCESSED AND CONVERTED INTO A SUITABLE FORMAT FOR THE COMPUTER TO USE IT.

In [ ]:
# store the users' reviews in a variable
user_reviews = elect_data['reviews.text']

reviews_text = elect_data.iloc[: , [21]]

# The reviews text are contained in the 21st column of the dataset so all the other columns will be dropped
print(reviews_text)


In [94]:
# create two files to store the input dataset and resulting output dataset
data_input = "electronics_dataset.csv"
data_output = "output_data.csv"

# create a new dataframe with only the reviews text column and store it in a csv file
df = elect_data.loc[: , ["reviews.text"]]
df.to_csv('output_data.csv', index=False)




# cols_to_remove = [0, 1, 2, 3, 4, 5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,22,23,24,25,26]

# cols_to_remove = sorted(cols_to_remove, reverse=True) # Reverse so we remove from the end first
# row_count = 0 # Current amount of rows processed

# with open(data_input, "r") as source:
#     reader = csv.reader(source)
#     with open(data_output, "w", newline='') as result:
#         writer = csv.writer(result)
#         for row in reader:
#             row_count += 1
#            # print('\r{0}'.format(row_count), end='') # Print rows processed
#             for col_index in cols_to_remove:
#                 del row[col_index]
#             writer.writerow(row)



In [ ]:
# CONVERT THE REVIEWS CSV FILE INTO A TEXT FILE

In [ ]:
with open ("reviews.txt", "w", encoding="utf-8") as new_file:
    writer = csv.writer(new_file)
    # for val in df["reviews.text"]:
    #     writer.writerow([val])
    
    with open("output_data.csv", encoding="utf-8") as old_file:
        for row in csv.reader(old_file):
            writer.writerow(row)
    
    # with open("output_data.csv") as old_file:
    #     [new_file.write(" ".join(row)+'\n') for row in csv.reader(old_file)]

    # new_file.close()

TO MAKE THE REVIEWS TEXT MORE MEANINGFUL TO A MACHINE, THE STOPWORDS ARE REMOVED USING THE NLTK LIBRARY AS THEY ADD LITTLE MEANING TO THE OVERALL TEXT.

In [ ]:
stop_words = set(stopwords.words('english'))

with open("output_data.csv", "r", encoding="utf-8") as csv_file, open("stpw_deleted.csv", "w", newline='', encoding="utf-8") as stop_word_file:
    word_writer = csv.writer(stop_word_file)
    word_reader = csv.reader(csv_file, delimiter=",")
    stpw_counter = 0
    for lines in word_reader:
        for txt_line in lines:
            txt_words = txt_line.split()
            filtered_words = [w for w in txt_words if w.lower() not in stop_words]
            stpw_counter += len(txt_words) - len(filtered_words)
            word_writer.writerow(filtered_words)
       
print("The number of stopwords in the dataset is: ", stpw_counter)


THE NEXT STEP IS TO SHARE THE SENTENCES OF THE REVIEWS INTO DIFFERENT PARTS OF SPEECH USING NLTK'S PARTS OF SPEECH TAGGER(POS TAGGING). THIS WILL HELP THE MACHINE TO UNDERSTAND THE MEANING OF THE SENTENCES BETTER 

In [ ]:
with open('output_data.csv', "r", encoding="utf-8") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for tokens in csv_reader: 
        print("POS::")
        tagged_tokens = nltk.pos_tag(tokens)
        print(tagged_tokens)

TO MAKE THE WORDS IN THE REVIEWS TEXT EVEN MORE MEANINGFUL TO THE MACHINE, WE USE STEMMING TO REDUCE THE WORDS TO THEIR ROOT FORMS. STEMMING IS A PROCESS OF REDUCING THE WORDS TO THEIR ROOT FORMS BY REMOVING THE ENDINGS OF THE WORDS. FOR EXAMPLE, THE WORDS 'RUNNING', 'RUNS', 'RUNNER' ARE ALL REDUCED TO THE ROOT FORM 'RUN'. THIS WILL HELP THE MACHINE TO UNDERSTAND THE MEANING OF THE WORDS BETTER.

In [7]:
# create a Porter Stemmer object
# porter = PorterStemmer()

# with open('electronics_dataset.csv', "r", encoding="utf-8") as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=',')
#     print("Porter Stemmer::")
#     for lines in csv_reader: 
#         for word in lines:
#             print(word, porter.stem(word))
d=0
porter = PorterStemmer()
with open('electronics_dataset.csv', "r", encoding="utf-8") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for lines in csv_reader: 
        txt_line = csv_file.read()
        txt_words = txt_line.split()
print("Porter Stemmer::")
for check_word in txt_words:
    print(check_word,porter.stem(check_word))

# porter = PorterStemmer()

# with open('electronics_dataset.csv', "r", encoding="utf-8") as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=',')
#     for lines in csv_reader: 
#         word_tokens = word_tokenize(lines[0])
#         for word in word_tokens:
#             print(word, porter.stem(word))


In [ ]:
# store the pool of english stopwords in a variable
# stop_words = set(stopwords.words('english'))

# # read the reviews text csv file 
# with open ("output_data.csv", "r", encoding="utf-8") as csv_file:
#     word_reader = csv.reader(csv_file, delimiter=",")
#     for lines in word_reader:
#         txt_line = csv_file.read()
#         txt_words = txt_line.split()
        
# stpw_counter = 0
        
# # add all words that are not stopwords to a list      
# for w in txt_words:
#     if w.lower() not in stop_words:
#         stop_word_file = open("stpw_deleted.csv", "a")
#         stop_word_file.write(" " + w)
#         stop_word_file.close()
#     else:
#         stpw_counter += 1
       
# print("The number of stopwords in the dataset is: ", stpw_counter)


THE NEXT STEP IS TO DROP THE REVIEWS COLUMN FROM THE DATSET AS IT IS NOT REQUIRED FOR BUILDING THE RECOMMENDER SYSTEM BASED ON COLLABORATIVE FILTERING FROM SILMILAR USER REVIEWS

In [ ]:
elect_data